In [37]:
import pandas as pd

In [38]:
path_to_mpa_output="/shared/projects/mudis4ls_is4_benchmark/test_results/metaphlan/metaphlan_output_gtdb.txt"
path_to_sylph_output="/shared/projects/mudis4ls_is4_benchmark/test_results/sylph/ERS12377136.fastq.gz.sylphmpa"

In [39]:
# Charger les fichiers en DataFrames
df_metaphlan = pd.read_csv(path_to_mpa_output, sep="\t", skiprows=1 )
df_sylph = pd.read_csv(path_to_sylph_output, sep="\t", skiprows=1)

In [40]:
# Modifier le nom de la première colonne
df_metaphlan = df_metaphlan.rename(columns={df_metaphlan.columns[0]: df_metaphlan.columns[0][1:]})

print(df_metaphlan.head())

                      clade_name  relative_abundance
0                    d__Bacteria           100.00000
1    d__Bacteria;p__Bacteroidota            52.20608
2    d__Bacteria;p__Firmicutes_A            43.38615
3    d__Bacteria;p__Firmicutes_C             3.35772
4  d__Bacteria;p__Proteobacteria             1.05005


In [41]:
# Renommer la colonne relative_abundance pour chaque outil
df_metaphlan.rename(columns={"relative_abundance": "abundance_metaphlan"}, inplace=True)
df_sylph.rename(columns={"relative_abundance": "abundance_sylph"}, inplace=True)


In [42]:
# Remplacement efficace avec regex
df_metaphlan["clade_name"] = df_metaphlan["clade_name"].str.replace(
    r"Firmicutes", "Bacillota", regex=True
).str.replace(
    r"Proteobacteria", "Pseudomonadota", regex=True
)

In [43]:
df_metaphlan["clade_name"] = df_metaphlan["clade_name"].str.replace(";", "|")


In [44]:
df_merged = df_metaphlan.merge(df_sylph, on="clade_name", how="outer")


In [45]:
df_merged

,clade_name,abundance_metaphlan,abundance_sylph,sequence_abundance,ANI (if strain-level),Coverage (if strain-level)
0,d__Bacteria,100.00000,100.0000,100.0001,NaN,NaN
1,d__Bacteria|p__Bacillota_A,43.38615,41.4963,36.1829,NaN,NaN
2,d__Bacteria|p__Bacillota_A|c__Clostridia,43.38615,41.4963,36.1829,NaN,NaN
3,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,22.7149,NaN,NaN
4,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,22.7149,NaN,NaN
...,...,...,...,...,...,...
96,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN
97,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN
98,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN
99,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN


In [46]:
df_merged["diff_abundance"] = df_merged["abundance_metaphlan"] - df_merged["abundance_sylph"]


In [47]:
df_merged

,clade_name,abundance_metaphlan,abundance_sylph,sequence_abundance,ANI (if strain-level),Coverage (if strain-level),diff_abundance
0,d__Bacteria,100.00000,100.0000,100.0001,NaN,NaN,0.00000
1,d__Bacteria|p__Bacillota_A,43.38615,41.4963,36.1829,NaN,NaN,1.88985
2,d__Bacteria|p__Bacillota_A|c__Clostridia,43.38615,41.4963,36.1829,NaN,NaN,1.88985
3,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,22.7149,NaN,NaN,-0.90287
4,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,22.7149,NaN,NaN,-0.90287
...,...,...,...,...,...,...,...
96,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN,NaN
97,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN,NaN
98,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN,NaN
99,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,0.9629,NaN,NaN,NaN


In [48]:
df_merged = df_merged[["clade_name", "abundance_metaphlan", "abundance_sylph","diff_abundance"]]


In [49]:
df_merged

,clade_name,abundance_metaphlan,abundance_sylph,diff_abundance
0,d__Bacteria,100.00000,100.0000,0.00000
1,d__Bacteria|p__Bacillota_A,43.38615,41.4963,1.88985
2,d__Bacteria|p__Bacillota_A|c__Clostridia,43.38615,41.4963,1.88985
3,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,-0.90287
4,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,-0.90287
...,...,...,...,...
96,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,NaN
97,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,NaN
98,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,NaN
99,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,NaN


In [50]:
output_dir='/shared/projects/mudis4ls_is4_benchmark/test_results/compare_mpa_sylph/'
df_merged.to_csv(output_dir+'compare_mpa_sylph.tsv',sep='\t',index=False)


In [51]:

# Species	
# Genus	
# Family	
# Order	
# Class
# Domain : bacteria

In [52]:
clade=df_merged["clade_name"]


In [53]:
clade

0                                            d__Bacteria
1                             d__Bacteria|p__Bacillota_A
2               d__Bacteria|p__Bacillota_A|c__Clostridia
3      d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
4      d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
                             ...                        
96     d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...
97     d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...
98     d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...
99     d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...
100    d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...
Name: clade_name, Length: 101, dtype: object

In [61]:
# Assurez-vous que la fonction renvoie un tuple ou une liste pour chaque ligne
def get_last_taxonomic_level(clade):
    parts = clade.split('|')
    last = parts[-1] if parts else None
    print(last.startswith("d__"), last)  # Pour déboguer et voir ce qui est retourné
    if last and last.startswith("d__"):
        return "domain", last
    elif last and last.startswith("p__"):
        return "phylum", last
    elif last and last.startswith("c__"):
        return "class", last
    elif last and last.startswith("o__"):
        return "order", last
    elif last and last.startswith("f__"):
        return "family", last
    elif last and last.startswith("g__"):
        return "genus", last
    elif last and last.startswith("s__"):
        return "species", last
    else:
        return "unknown", last

# Appliquer la fonction à la colonne "clade_name" et obtenir deux colonnes
df_merged[['taxonomic_level', 'last_taxon']] = df_merged['clade_name'].apply(
    lambda x: pd.Series(get_last_taxonomic_level(x))
)

# Vérifiez si les colonnes sont remplies correctement
print(df_merged[['taxonomic_level', 'last_taxon']].head())


True d__Bacteria
False p__Bacillota_A
False c__Clostridia
False o__Lachnospirales
False f__Lachnospiraceae
False g__Agathobacter
False s__Agathobacter rectalis
False t__GCF_000020605.1
False g__Blautia
False g__Blautia_A
False s__Blautia_A wexlerae
False t__GCF_025148125.1
False s__Blautia hansenii
False t__GCF_002222595.2
False g__Brotaphodocola
False s__Brotaphodocola sp900540335
False t__GCA_900540335.1
False g__Enterocloster
False s__Enterocloster sp001517625
False t__GCF_001517625.2
False g__Lachnospira
False s__Lachnospira sp000437735
False t__GCF_020564355.1
False g__Mediterraneibacter
False s__Mediterraneibacter faecis
False s__Mediterraneibacter torques
False t__GCF_000153925.1
False g__Roseburia
False s__Roseburia intestinalis
False t__GCF_900537995.1
False g__Ventrimonas
False s__Ventrimonas sp900540335
False o__Oscillospirales
False f__Acutalibacteraceae
False g__Fimivicinus
False s__Fimivicinus sp900544715
False t__GCA_900544715.1
False g__Ruminococcus_E
False s__Ruminococ

/tmp/ipykernel_658904/3983686473.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_merged[['taxonomic_level', 'last_taxon']] = df_merged['clade_name'].apply(


In [62]:
df_merged

,clade_name,abundance_metaphlan,abundance_sylph,diff_abundance,taxonomic_level,last_taxon
0,d__Bacteria,100.00000,100.0000,0.00000,domain,d__Bacteria
1,d__Bacteria|p__Bacillota_A,43.38615,41.4963,1.88985,phylum,p__Bacillota_A
2,d__Bacteria|p__Bacillota_A|c__Clostridia,43.38615,41.4963,1.88985,class,c__Clostridia
3,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,-0.90287,order,o__Lachnospirales
4,d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...,23.22263,24.1255,-0.90287,family,f__Lachnospiraceae
...,...,...,...,...,...,...
96,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,NaN,order,o__Burkholderiales
97,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,NaN,family,f__Burkholderiaceae
98,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,NaN,genus,g__Sutterella
99,d__Bacteria|p__Pseudomonadota|c__Gammaproteoba...,NaN,1.3124,NaN,species,s__Sutterella wadsworthensis_A


In [63]:
output_dir='/shared/projects/mudis4ls_is4_benchmark/test_results/compare_mpa_sylph/'
df_merged.to_csv(output_dir+'compare_mpa_sylph.tsv',sep='\t',index=False)


In [30]:


# Supposons que votre DataFrame s'appelle df
result = df_merged[df_merged['diff_abundance'].notna()]
print(result)


                                           clade_name  abundance_metaphlan  \
0                                         d__Bacteria            100.00000   
1                          d__Bacteria|p__Bacillota_A             43.38615   
2            d__Bacteria|p__Bacillota_A|c__Clostridia             43.38615   
3   d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...             23.22263   
4   d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...             23.22263   
5   d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...             16.27582   
6   d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...             16.27582   
17  d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...              1.94173   
18  d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...              1.94173   
23  d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...              1.49431   
25  d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...              1.25307   
27  d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...           

In [37]:
output_dir='/shared/projects/mudis4ls_is4_benchmark/test_results/compare_mpa_sylph/'
result.to_csv(output_dir+'compare_mpa_sylph.csv',index=False)


In [36]:
output_dir='/shared/projects/mudis4ls_is4_benchmark/test_results/compare_mpa_sylph/'
result.to_csv(output_dir+'compare_mpa_sylph.tsv',sep='\t',index=False)


In [28]:
resultat = df_merged[df_merged['diff_abundance'].isna()]['clade_name']
print(resultat)

7      d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
8      d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
9      d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
10     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
11     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
12     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
13     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
14     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
15     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
16     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
19     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
20     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
21     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
22     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
24     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
26     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
29     d__Bacteria|p__Bacillota_A|c__Clostridia|o__La...
30     d__Bacteria|p__Bacillota

In [11]:
#Pseudomonadota – anciennement Proteobacteria 
#Bacillota – anciennement Firmicutes
# Regarde ce qui se passe si on change juste l'assignation de ces 2 clades

In [27]:
import seaborn as sns
import matplotlib.pyplot as plt

In [12]:
sns.heatmap(df.set_index("clade_name"), annot=True, cmap="coolwarm", linewidths=0.5)
plt.title("Comparaison des abondances par clade et outil")
plt.show()
